In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
PATH = 'cartosat2series_2018_f.jpg'

In [5]:
OUT = './cropped/'

In [3]:
img = cv2.imread(PATH)

In [4]:
img.shape

(9154, 24780, 3)

In [6]:
PATCH_SIZE = 600

In [8]:
height, width = img.shape[:2]

In [ ]:
# its grayscale so ditch 3 channels

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# lets see a sample

In [ ]:
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
show(img[:1000,:1000])

In [ ]:
patches = []

In [ ]:
for i in range(height//PATCH_SIZE):
    for j in range(width//PATCH_SIZE):
        patch = np.array(img[i*PATCH_SIZE:(i+1)*PATCH_SIZE, j*PATCH_SIZE:(j+1)*PATCH_SIZE], dtype=np.float64)
        mn, sd = np.mean(patch), np.std(patch)
        patch = (patch - mn)/(sd+0.001)
        patches.append(patch)

In [ ]:
show(patches[0])

In [ ]:
patches = np.array(patches)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=100)
pca_patches = pca.fit_transform(patches.reshape(-1, PATCH_SIZE**2))

In [ ]:
patches.shape

In [ ]:
pca_patches.shape

In [ ]:
# now apply kmeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=2018)

In [ ]:
kmeaned = kmeans.fit(pca_patches)

In [ ]:
show(patches[0])

In [ ]:
show(patches[2])

In [ ]:
np.unique(kmeaned.labels_)

In [ ]:
labels = kmeaned.labels_

In [ ]:
overlay = np.ones((height//PATCH_SIZE, width//PATCH_SIZE))
overlay = np.ones((height, width))

In [ ]:
color = [(0,0,0), (0,1,0), (0,0,1), (1,0,0)]
color = [0, 64, 128, 256][::-1]

In [ ]:
colors = []

In [ ]:
for i in range(height//PATCH_SIZE):
    for j in range(width//PATCH_SIZE):
        pixel = color[labels[i*(width//PATCH_SIZE) + j]]
#         overlay[i, j] = np.random.choice(color)
        overlay[i*PATCH_SIZE:(i+1)*PATCH_SIZE, j*PATCH_SIZE:(j+1)*PATCH_SIZE] = np.random.choice(color)

#         overlay[i, j, 1] = pixel[1]
#         overlay[i, j, 2] = pixel[2]
        colors.append(pixel)

In [ ]:
# plt.imshow(overlay)

In [ ]:
overlay.shape

In [ ]:
colors

In [9]:
for i in range(height//PATCH_SIZE):
    for j in range(width//PATCH_SIZE):
        patch = np.array(img[i*PATCH_SIZE:(i+1)*PATCH_SIZE, j*PATCH_SIZE:(j+1)*PATCH_SIZE,:])
        cv2.imwrite(OUT+'{0}_{1}.jpg'.format(i,j), patch)

In [10]:
# cv2.imwrite('overlay.jpg', overlay)